In [1]:
import os
import pandas as pd
import bs4
import requests
from bs4 import BeautifulSoup
from bs4 import NavigableString
from bs4 import Tag
from bs4 import Comment
from bs4 import PageElement
from pprint import pprint
import re

In [2]:
path = "linkedin_page/scraps/02/"

qtd_scraps = len(os.listdir(path))
soup_list = []
for i in range(qtd_scraps):
    filename = "post_" + str(i) + ".html"
    with open(path + filename, 'r', encoding='utf8') as file:
        soup_list.append(BeautifulSoup(file, 'html.parser'))


In [3]:
print(len(soup_list))

102


In [4]:
post_columns = [
    "date",
    "shared_post"
    "type",
    "url",
    "text_content",
    "qtd_reactions",
    "qtd_comments",
    "qtd_shares",
]
df = pd.DataFrame(columns=post_columns)

def add_row(df, post_dict):
    new_row = pd.Series(post_dict, index=df.columns)
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return df

def remove_last_row(df):
    df = df.drop(df.index[-1])
    return df

In [5]:
df_dict = {
    "publish_date": [],
    "shared_post": [],
    "author_name": [],
    "author_profile_url": [],
    "post_type": [],
    "post_url": [],
    "post_content": [],
    "reaction_count": [],
    "comment_count": [],
    "share_count": [],
}
corr_index_filename = []


for index, soup in enumerate(soup_list):
    # print(index)
    df_dict["publish_date"].append(
        soup.find_all(class_="update-components-actor__meta")[0]
        .find_all(class_="visually-hidden")[-1]
        .text.split("•")[0]
        if soup.find_all(class_="update-components-actor__meta") != []
        else None
        if (soup.find(class_="update-components-promo-v1__text-container") != None)
        or (len(soup.find_all(tag="div")) <= 7)
        else "Unknown"
    )

    # Linkedin Promo or Unloaded scraps
    if df_dict["publish_date"][-1] == None:
        df_dict["publish_date"].pop()
        continue
    corr_index_filename.append(soup_list[index])

    df_dict["shared_post"].append(
        True
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else False
    )

    author_name = soup.find(class_="update-components-actor__name").span.span.text
    df_dict["author_name"].append(author_name)

    author_profile_url = soup.find(class_="update-components-actor__meta-link")["href"]
    df_dict["author_profile_url"].append(author_profile_url)

    df_dict["post_type"].append(
        "Shared Content"
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else "Poll"
        if soup.find(class_="update-components-poll") is not None
        else "Scheduled Event"
        if soup.find(class_="update-components-scheduled-live-content__event-link")
        is not None
        else "Live"
        if soup.find(class_="video-live") is not None
        else "Single Image"
        if soup.find(class_="update-components-image--single-image") is not None
        else "Text only"
        if "update-v2-social-activity"
        in soup.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
        else "Default"
    )

    data_urn = soup.find(class_="feed-shared-update-v2")["data-urn"]

    df_dict["post_url"].append("https://www.linkedin.com/feed/update/" + data_urn)

    df_dict["post_content"].append(
        soup.find(class_="update-components-update-v2__commentary").span.span.text
        if df_dict["post_type"][-1] == "Shared Content"
        else soup.find(class_="update-components-text").span.span.text
    )

    # Reaction
    if soup.find(class_="social-details-social-counts__reactions-count") != None:
        df_dict["reaction_count"].append(
            soup.find(
                class_="social-details-social-counts__reactions-count"
            ).text.strip()
        )

    else:
        qtd_reactions = soup.find(
            class_="social-details-social-counts__social-proof-text"
        ).text.strip()

        re_pattern_reactions = r"\b(\d+)\s+pessoas\b"

        df_dict["reaction_count"].append(
            1 + int(re.findall(re_pattern_reactions, qtd_reactions)[0])
        )

    # Comments
    comment_count = (
        soup.find(
            class_="social-details-social-counts__comments"
        ).button.span.text.strip()
        if soup.find(class_="social-details-social-counts__comments") != None
        else "0"
    )

    re_pattern_comments = r"\b(\d+)\b" if comment_count != "0" else "0"

    df_dict["comment_count"].append(re.findall(re_pattern_comments, comment_count)[0])

    # Share
    df_dict["share_count"].append(
        0
        if len(
            soup.find_all(class_="social-details-social-counts__item--right-aligned")
        )
        <= 1
        else soup.find_all(class_="social-details-social-counts__item--right-aligned")[
            1
        ]
        .button.span.text.strip()
        .split(" ")[0]
    )

df = pd.DataFrame(df_dict)

In [7]:
df

,publish_date,shared_post,author_name,author_profile_url,post_type,post_url,post_content,reaction_count,comment_count,share_count
0,23 h,False,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Single Image,https://www.linkedin.com/feed/update/urn:li:ac...,"Igor GonçalvesÉ muuuita emoção, você fez seu c...",83,16,2
1,5 d,True,João Victor Pereira dos Santos,https://www.linkedin.com/in/joaovictorps?miniP...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,"Os trabalhos na SouJunior não param, recenteme...",37,8,1
2,1 sem,False,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Live,https://www.linkedin.com/feed/update/urn:li:ac...,SouJunior: The Champions discord: https://lnkd...,25,18,0
3,2 sem,True,Dener Garcia 🧑🏻‍💻,https://www.linkedin.com/in/denergarcia?miniPr...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Estou participando do desafio da SouJunior ont...,27,0,0
4,3 sem,True,Pedro Gomes,https://www.linkedin.com/in/pgomes2g?miniProfi...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Já se perguntaram como fica um projeto grande ...,142,10,2
...,...,...,...,...,...,...,...,...,...,...
88,1 m,True,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Time bora colar em peso! Se network funciona? ...,63,4,0
89,2 m,False,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Live,https://www.linkedin.com/feed/update/urn:li:ac...,Live MVCNos ajude com $: Github: https://lnkd....,24,25,0
90,1 m,True,Daiane Bolzan,https://www.linkedin.com/in/daiane-deponti-bol...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,Sábado é dia de reunião com a equipe mais anim...,129,4,8
91,1 m,False,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Live,https://www.linkedin.com/feed/update/urn:li:ac...,Live: aprender GOLang live #4 !Nos ajude com $...,19,6,0


In [25]:
def export_html_and_view_link(df_index):   
    with open(f'linkedin_page/scraps/post_preview.html', 'w', encoding='utf-8') as file:
            file.write(corr_index_filename[df_index].prettify())
    print(df.iloc[df_index, 4])

    return corr_index_filename[df_index]

preview_soup = export_html_and_view_link(1)

https://www.linkedin.com/feed/update/urn:li:activity:7134987137451458562


In [30]:
# extract preview
author_name = preview_soup.find(class_="update-components-actor__name").span.span.text
author_profile_url = preview_soup.find(class_="update-components-actor__meta-link")['href']

'https://www.linkedin.com/in/joaovictorps?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAACqkFp0B84HNrtTMloucqGrLz_YAFjVkBmE'

In [ ]:
df.to_csv("linkedin_page/dataframes/02-dataframe.csv", index=False)